# Check Data Availability
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* May 13, 2022

This notebooks checks to see what years of data are available from a specified stored proc, and team database.

In [1]:
import boto3 
import pandas as pd
import psycopg2

from shared_utilities import helpers

### Create a connection to RedShift PROD:

In [2]:
if not boto3.DEFAULT_SESSION:
    ! aws sso login --profile Stellaralgo-DataScienceAdmin
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

CWTQ-DPKV
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [3]:
CLUSTER = "qa-app"
DBNAME = "stlr"
LKUPCLIENTID = 50

CNXN = helpers.get_redshift_connection(CLUSTER, DBNAME)

### Run the stored proc to get retention training and inference datasets:

In [4]:
#sql = f"CALL modeldata(16, 2010, 2022, 'temp_cursor')"
sql = f"CALL ds.getproductpropensitymodeldata({LKUPCLIENTID}, 2010, 2022, 'temp_cursor')"
#sql = f"CALL modeldata(16, 2010, 2022, 'temp_cursor')"

cur = CNXN.cursor()

try:
    cur.execute(sql)

    named_cursor = CNXN.cursor('temp_cursor')
    data = named_cursor.fetchall()

    cols = [row[0] for row in named_cursor.description]
    df = pd.DataFrame(data=data, columns=cols)

    CNXN.commit()

except Exception as e:
    print(f"ERROR:, {e}")
    cur.execute("ROLLBACK")

finally:
    cur.close()

### Check how much data we have for each season:

In [13]:
if df.shape[0] > 0:
    
    print(f"DATA AVAILABILITY REPORT:")
    print(f" > {CLUSTER} {DBNAME}")
    df["year"].value_counts().sort_index(ascending=False)

else:
    print(f"NO ROWS AVAILABLE: {df.shape[0]} !")


NO ROWS AVAILABLE: 0 !


### Done